# Step 3: Mini GPT — 从 Decoder-Only 到 LLaMA

> **目标**：理解 NTP 训练 + LLM 常见改动，为看懂 MiniOneRec 的 sft.py / rl.py 打基础
>
> **前置**：Step 2 已学完 Transformer、RoPE、Causal Mask
>
> **参考书目**：复旦大学《大规模语言模型：从理论到实践》第二版 · 第 2 章

### 📖 书本章节对照表

| 书本章节 | 页码 | Notebook 对应 |
|---|---|---|
| 2.1.1 嵌入表示层 | p18 | Section 3: Token Embedding |
| 2.1.2 注意力层 | p19 | Section 3/7: MHA → GQA |
| 2.1.3 前馈层 | p23 | Section 6: ReLU FFN → SwiGLU |
| 2.1.4 残差连接与层归一化 | p23 | Section 5: LayerNorm → RMSNorm |
| 2.1.5 编码器和解码器结构 | p25 | Section 1: Encoder-Decoder vs Decoder-Only |
| 2.2.1 自监督预训练 | p30 | Section 2: NTP 训练目标 |
| 2.2.3 预训练语言模型实践 | p32 | Section 4: 字符级语言模型训练 |
| 2.3.1 LLaMA 的模型结构 | p41 | Section 8: MiniLLaMA 整合 |
| 2.3.2 注意力机制优化 | p49 | Section 7: GQA + Section 11.5: MLA |
| 2.4 混合专家模型 | p57 | Section 12 对比表（MoE 留到 Step 4）|

---

## Part 1: Decoder-Only 基础

## 1. 从 Encoder-Decoder 到 Decoder-Only

**Step 2 的完整 Transformer：**
```
Encoder (Self-Attention + FFN) × N
    ↓ memory
Decoder (Masked Self-Attn + Cross-Attn + FFN) × N
    ↓
Output
```

**GPT 的 Decoder-Only：**
```
Decoder (Masked Self-Attention + FFN) × N
    ↓
Output
```

| 区别 | Encoder-Decoder | Decoder-Only (GPT) |
|------|----------------|---------------------|
| Encoder | ✅ 有 | ❌ 去掉 |
| Cross-Attention | ✅ 有 | ❌ 去掉 |
| Self-Attention | 双向(Encoder) + 因果(Decoder) | 只有因果(Causal) |
| 典型任务 | 翻译、摘要 | 文本生成、对话 |

**核心思想**：只保留 Causal Masked Self-Attention + FFN，一个 token 只能看到它前面的 token。

## 2. NTP: Next Token Prediction

**核心概念**：输入右移一位就是标签

```
输入: [A, B, C, D]    → 模型预测每个位置的"下一个 token"
标签: [B, C, D, E]    → 就是输入往后移一位

Loss = CrossEntropy(预测, 标签)  对每个位置都算，然后取平均
```

这和 Step 1 的 CE Loss 完全一样，只是现在对序列中**每个位置**都算 loss。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time

# === NTP: Next Token Prediction 演示 ===
# 假设 vocab: A=0, B=1, C=2, D=3, E=4
sequence = torch.tensor([0, 1, 2, 3, 4])  # [A, B, C, D, E]

# 输入 = 前 n-1 个 token，标签 = 后 n-1 个 token
inputs = sequence[:-1]   # [A, B, C, D]
labels = sequence[1:]    # [B, C, D, E]

print(f"输入: {inputs.tolist()}  (A=0, B=1, C=2, D=3)")
print(f"标签: {labels.tolist()}  (B=1, C=2, D=3, E=4)")
print(f"\n每个位置的任务:")
for i in range(len(inputs)):
    print(f"  位置 {i}: 看到 {inputs[:i+1].tolist()} → 预测 {labels[i].item()}")

# 模拟模型输出 logits: (seq_len, vocab_size)
vocab_size = 5
logits = torch.randn(4, vocab_size)

# CE Loss
loss = F.cross_entropy(logits, labels)
print(f"\nCE Loss = {loss.item():.4f}")
print(f"理论随机 loss = -ln(1/{vocab_size}) = {math.log(vocab_size):.4f}")

输入: [0, 1, 2, 3]  (A=0, B=1, C=2, D=3)
标签: [1, 2, 3, 4]  (B=1, C=2, D=3, E=4)

每个位置的任务:
  位置 0: 看到 [0] → 预测 1
  位置 1: 看到 [0, 1] → 预测 2
  位置 2: 看到 [0, 1, 2] → 预测 3
  位置 3: 看到 [0, 1, 2, 3] → 预测 4

CE Loss = 1.9544
理论随机 loss = -ln(1/5) = 1.6094


## 3. 手写 Mini GPT 模型

复用 Step 2 的组件思路，但简化为 Decoder-Only：
- ✅ Causal Mask（Step 2 第5节）
- ✅ RoPE（Step 2 第12节）
- ❌ 去掉 Encoder、Cross-Attention
- ➕ 加上 Token Embedding + 输出投影层

In [2]:
# === 基础组件 ===

def precompute_rope_freqs(dim, max_seq_len, base=10000.0):
    """预计算 RoPE 频率 (复用 Step 2 第12节)"""
    freqs = 1.0 / (base ** (torch.arange(0, dim, 2).float() / dim))
    positions = torch.arange(max_seq_len).float()
    angles = torch.outer(positions, freqs)  # (seq_len, dim/2)
    return torch.cos(angles), torch.sin(angles)

def apply_rope(x, cos, sin):
    """对 x 应用 RoPE: x shape = (B, n_heads, T, head_dim)"""
    seq_len = x.shape[-2]
    cos = cos[:seq_len].unsqueeze(0).unsqueeze(0)  # (1,1,T,dim/2)
    sin = sin[:seq_len].unsqueeze(0).unsqueeze(0)
    x1, x2 = x[..., :x.shape[-1]//2], x[..., x.shape[-1]//2:]
    return torch.cat([x1 * cos - x2 * sin, x1 * sin + x2 * cos], dim=-1)

class MultiHeadAttention(nn.Module):
    """多头因果自注意力 + RoPE"""
    def __init__(self, model_dim, n_heads, max_seq_len=512):
        super().__init__()
        self.n_heads = n_heads
        self.head_dim = model_dim // n_heads
        self.Wq = nn.Linear(model_dim, model_dim, bias=False)
        self.Wk = nn.Linear(model_dim, model_dim, bias=False)
        self.Wv = nn.Linear(model_dim, model_dim, bias=False)
        self.Wo = nn.Linear(model_dim, model_dim, bias=False)
        cos, sin = precompute_rope_freqs(self.head_dim, max_seq_len)
        self.register_buffer('cos', cos)
        self.register_buffer('sin', sin)

    def forward(self, x, mask=None):
        B, T, D = x.shape
        q = self.Wq(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = self.Wk(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        v = self.Wv(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        q = apply_rope(q, self.cos, self.sin)
        k = apply_rope(k, self.cos, self.sin)
        scale = math.sqrt(self.head_dim)
        scores = torch.matmul(q, k.transpose(-2, -1)) / scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = F.softmax(scores, dim=-1)
        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).contiguous().view(B, T, D)
        return self.Wo(out)

class FFN(nn.Module):
    """前馈网络 (ReLU 版本，后面会升级为 SwiGLU)"""
    def __init__(self, model_dim, ff_dim=None):
        super().__init__()
        ff_dim = ff_dim or model_dim * 4
        self.w1 = nn.Linear(model_dim, ff_dim, bias=False)
        self.w2 = nn.Linear(ff_dim, model_dim, bias=False)

    def forward(self, x):
        return self.w2(F.relu(self.w1(x)))

class TransformerBlock(nn.Module):
    """Decoder Block: Pre-Norm → Attention → Residual → Pre-Norm → FFN → Residual"""
    def __init__(self, model_dim, n_heads, max_seq_len=512):
        super().__init__()
        self.norm1 = nn.LayerNorm(model_dim)
        self.attn = MultiHeadAttention(model_dim, n_heads, max_seq_len)
        self.norm2 = nn.LayerNorm(model_dim)
        self.ffn = FFN(model_dim)

    def forward(self, x, mask=None):
        x = x + self.attn(self.norm1(x), mask)
        x = x + self.ffn(self.norm2(x))
        return x

class MiniGPT(nn.Module):
    """Mini GPT: Token Embedding + N × DecoderBlock + LM Head"""
    def __init__(self, vocab_size, model_dim=64, n_heads=4, n_layers=2, max_seq_len=128):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, model_dim)
        self.blocks = nn.ModuleList([
            TransformerBlock(model_dim, n_heads, max_seq_len) for _ in range(n_layers)
        ])
        self.norm = nn.LayerNorm(model_dim)
        self.lm_head = nn.Linear(model_dim, vocab_size, bias=False)
        self.max_seq_len = max_seq_len

    def forward(self, idx, targets=None):
        B, T = idx.shape
        x = self.tok_emb(idx)  # (B, T, D)
        mask = torch.tril(torch.ones(T, T, device=idx.device)).unsqueeze(0).unsqueeze(0)
        for block in self.blocks:
            x = block(x, mask)
        x = self.norm(x)
        logits = self.lm_head(x)  # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

# 测试
model = MiniGPT(vocab_size=100, model_dim=64, n_heads=4, n_layers=2)
x = torch.randint(0, 100, (2, 10))
logits, _ = model(x)
print(f"输入 shape: {x.shape}")
print(f"输出 logits shape: {logits.shape}")  # (2, 10, 100)
print(f"模型参数量: {sum(p.numel() for p in model.parameters()):,}")

输入 shape: torch.Size([2, 10])
输出 logits shape: torch.Size([2, 10, 100])
模型参数量: 111,744


## 4. 训练一个字符级语言模型

用一小段中文文本训练 MiniGPT，直观感受：
1. Loss 下降过程
2. 自回归生成（模型"续写"）

In [ ]:
# === 字符级语言模型训练 ===
text = "天地玄黄宇宙洪荒日月盈昃辰宿列张寒来暑往秋收冬藏闰余成岁律吕调阳云腾致雨露结为霜"

# 构建字符词表
chars = sorted(set(text))
vocab_size = len(chars)
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}
data = torch.tensor([char2idx[c] for c in text])

print(f"文本长度: {len(text)}, 词表大小: {vocab_size}")
print(f"词表: {''.join(chars)}")

# 构建训练数据: 滑动窗口
seq_len = 16
xs, ys = [], []
for i in range(len(data) - seq_len):
    xs.append(data[i:i+seq_len])
    ys.append(data[i+1:i+seq_len+1])
xs, ys = torch.stack(xs), torch.stack(ys)
print(f"训练样本数: {xs.shape[0]}, 序列长度: {seq_len}")

# 训练
model = MiniGPT(vocab_size=vocab_size, model_dim=64, n_heads=4, n_layers=2, max_seq_len=seq_len+1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(200):
    logits, loss = model(xs, ys)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 40 == 0:
        print(f"Epoch {epoch+1:3d} | Loss: {loss.item():.4f}")

# 自回归生成
def generate_greedy(model, start_ids, max_new=20):
    model.eval()
    idx = start_ids.unsqueeze(0)  # (1, T)
    with torch.no_grad():
        for _ in range(max_new):
            logits, _ = model(idx[:, -model.max_seq_len:])
            next_id = logits[:, -1, :].argmax(dim=-1, keepdim=True)
            idx = torch.cat([idx, next_id], dim=1)
    return idx[0].tolist()

print(f"\n=== 生成测试 ===")
for start in ["天地", "寒来", "云腾"]:
    ids = torch.tensor([char2idx[c] for c in start])
    out = generate_greedy(model, ids)
    print(f"  '{start}' → {''.join(idx2char.get(i, '?') for i in out)}")

### Part 1 小结

| 概念 | 要点 |
|------|------|
| Decoder-Only | 去掉 Encoder + Cross-Attention，只保留 Causal Self-Attention + FFN |
| NTP | `input[:-1]` 预测 `input[1:]`，每个位置都算 CE Loss |
| Causal Mask | 下三角矩阵，保证 token 只看到前面的 |
| RoPE | 旋转位置编码，不需要学习，外推性好 |
| 自回归生成 | 循环：预测下一个 token → 拼接 → 再预测 |

---

## Part 2: LLaMA 改动

现代 LLM（LLaMA、Qwen、DeepSeek）对原版 Transformer 做了几个关键改动。

## 5. RMSNorm 替代 LayerNorm

**为什么？** RMSNorm 去掉了均值计算，只保留方差归一化，速度更快，效果相当。

| | LayerNorm | RMSNorm |
|---|-----------|---------|
| 公式 | $(x - \mu) / \sigma \cdot \gamma + \beta$ | $x / \text{RMS}(x) \cdot \gamma$ |
| 均值 | ✅ 需要计算 | ❌ 不需要 |
| 偏置 | ✅ 有 β | ❌ 没有 |
| 速度 | 较慢 | 更快 |

In [ ]:
# === RMSNorm vs LayerNorm ===

class RMSNorm(nn.Module):
    """Root Mean Square Layer Normalization (LLaMA/Qwen 使用)"""
    def __init__(self, dim, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(dim))
        self.eps = eps

    def forward(self, x):
        rms = torch.sqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        return x / rms * self.weight

# 对比
x = torch.randn(2, 5, 64)
ln = nn.LayerNorm(64)
rmsn = RMSNorm(64)

out_ln = ln(x)
out_rms = rmsn(x)
print(f"输入 shape: {x.shape}")
print(f"LayerNorm 输出 mean: {out_ln.mean():.6f}, std: {out_ln.std():.4f}")
print(f"RMSNorm   输出 mean: {out_rms.mean():.6f}, std: {out_rms.std():.4f}")
print(f"\nLayerNorm 参数: weight + bias = {sum(p.numel() for p in ln.parameters())}")
print(f"RMSNorm   参数: weight only  = {sum(p.numel() for p in rmsn.parameters())}")

## 6. SwiGLU 替代 ReLU FFN

**为什么？** SwiGLU 提供更强的非线性能力，LLaMA / Qwen / DeepSeek 都在用。

| | ReLU FFN | SwiGLU FFN |
|---|---------|------------|
| 公式 | ReLU(xW₁)W₂ | (Swish(xW₁) ⊙ xW₃)W₂ |
| 门控 | ❌ | ✅ 有门控机制 |
| 隐藏层 | 4d | 8d/3（调整后参数量相当）|

In [ ]:
# === SwiGLU FFN ===

class SwiGLU_FFN(nn.Module):
    """SwiGLU Feed-Forward Network (LLaMA 风格)"""
    def __init__(self, model_dim, ff_dim=None):
        super().__init__()
        ff_dim = ff_dim or int(model_dim * 8 / 3)
        self.w1 = nn.Linear(model_dim, ff_dim, bias=False)  # gate
        self.w3 = nn.Linear(model_dim, ff_dim, bias=False)  # up
        self.w2 = nn.Linear(ff_dim, model_dim, bias=False)  # down

    def forward(self, x):
        return self.w2(F.silu(self.w1(x)) * self.w3(x))

# 对比参数量
d = 64
relu_ffn = FFN(d)
swiglu_ffn = SwiGLU_FFN(d)
print(f"ReLU FFN   参数量: {sum(p.numel() for p in relu_ffn.parameters()):,}")
print(f"SwiGLU FFN 参数量: {sum(p.numel() for p in swiglu_ffn.parameters()):,}")

x = torch.randn(2, 5, d)
print(f"\nReLU FFN   输出 shape: {relu_ffn(x).shape}")
print(f"SwiGLU FFN 输出 shape: {swiglu_ffn(x).shape}")

## 7. GQA: Grouped Query Attention

**演进路线**：MHA → MQA → GQA

| | MHA | MQA | GQA |
|---|-----|-----|-----|
| Q 头数 | H | H | H |
| K/V 头数 | H | 1 | G (1 < G < H) |
| 参数量 | 最多 | 最少 | 中间 |
| 效果 | 最好 | 略差 | 接近 MHA |
| 代表 | GPT-2 | PaLM | LLaMA-2/3, Qwen |

**核心思想**：多个 Q 头共享同一组 K/V 头，减少 KV Cache 显存。

In [ ]:
# === GQA: Grouped Query Attention ===

class GroupedQueryAttention(nn.Module):
    """GQA: n_kv_heads 组 K/V，每组被 n_heads/n_kv_heads 个 Q 头共享"""
    def __init__(self, model_dim, n_heads, n_kv_heads, max_seq_len=512):
        super().__init__()
        assert n_heads % n_kv_heads == 0
        self.n_heads = n_heads
        self.n_kv_heads = n_kv_heads
        self.n_rep = n_heads // n_kv_heads
        self.head_dim = model_dim // n_heads

        self.Wq = nn.Linear(model_dim, n_heads * self.head_dim, bias=False)
        self.Wk = nn.Linear(model_dim, n_kv_heads * self.head_dim, bias=False)
        self.Wv = nn.Linear(model_dim, n_kv_heads * self.head_dim, bias=False)
        self.Wo = nn.Linear(model_dim, model_dim, bias=False)
        cos, sin = precompute_rope_freqs(self.head_dim, max_seq_len)
        self.register_buffer('cos', cos)
        self.register_buffer('sin', sin)

    def forward(self, x, mask=None):
        B, T, _ = x.shape
        q = self.Wq(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = self.Wk(x).view(B, T, self.n_kv_heads, self.head_dim).transpose(1, 2)
        v = self.Wv(x).view(B, T, self.n_kv_heads, self.head_dim).transpose(1, 2)
        q = apply_rope(q, self.cos, self.sin)
        k = apply_rope(k, self.cos, self.sin)
        # 扩展 KV: (B, n_kv_heads, T, D) → (B, n_heads, T, D)
        k = k.repeat_interleave(self.n_rep, dim=1)
        v = v.repeat_interleave(self.n_rep, dim=1)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        out = torch.matmul(F.softmax(scores, dim=-1), v)
        out = out.transpose(1, 2).contiguous().view(B, T, -1)
        return self.Wo(out)

# 对比参数量
d, h = 64, 8
mha = MultiHeadAttention(d, h)
gqa = GroupedQueryAttention(d, n_heads=h, n_kv_heads=2)
print(f"MHA (8Q, 8KV) 参数量: {sum(p.numel() for p in mha.parameters()):,}")
print(f"GQA (8Q, 2KV) 参数量: {sum(p.numel() for p in gqa.parameters()):,}")
print(f"KV 参数节省: {1 - sum(p.numel() for p in gqa.parameters()) / sum(p.numel() for p in mha.parameters()):.1%}")

x = torch.randn(2, 10, d)
mask = torch.tril(torch.ones(10, 10)).unsqueeze(0).unsqueeze(0)
print(f"\nGQA 输出 shape: {gqa(x, mask).shape}")

## 8. 升级版 Mini GPT（LLaMA 风格）

把 Section 5/6/7 的改动整合：RMSNorm + SwiGLU + GQA + Pre-Norm

| 组件 | 原版 GPT | LLaMA 风格 |
|------|---------|------------|
| Norm | LayerNorm | RMSNorm |
| FFN | ReLU, 4d | SwiGLU, 8d/3 |
| Attention | MHA | GQA |
| Norm 位置 | Post-Norm 或 Pre-Norm | Pre-Norm |
| 位置编码 | 学习式 / Sinusoidal | RoPE |

In [ ]:
# === LLaMA 风格 Mini GPT ===

class LLaMABlock(nn.Module):
    """LLaMA Decoder Block: RMSNorm + GQA + SwiGLU"""
    def __init__(self, model_dim, n_heads, n_kv_heads, max_seq_len=512):
        super().__init__()
        self.norm1 = RMSNorm(model_dim)
        self.attn = GroupedQueryAttention(model_dim, n_heads, n_kv_heads, max_seq_len)
        self.norm2 = RMSNorm(model_dim)
        self.ffn = SwiGLU_FFN(model_dim)

    def forward(self, x, mask=None):
        x = x + self.attn(self.norm1(x), mask)
        x = x + self.ffn(self.norm2(x))
        return x

class MiniLLaMA(nn.Module):
    """LLaMA 风格的 Mini GPT"""
    def __init__(self, vocab_size, model_dim=64, n_heads=8, n_kv_heads=2,
                 n_layers=2, max_seq_len=128):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, model_dim)
        self.blocks = nn.ModuleList([
            LLaMABlock(model_dim, n_heads, n_kv_heads, max_seq_len)
            for _ in range(n_layers)
        ])
        self.norm = RMSNorm(model_dim)
        self.lm_head = nn.Linear(model_dim, vocab_size, bias=False)
        self.max_seq_len = max_seq_len

    def forward(self, idx, targets=None):
        B, T = idx.shape
        x = self.tok_emb(idx)
        mask = torch.tril(torch.ones(T, T, device=idx.device)).unsqueeze(0).unsqueeze(0)
        for block in self.blocks:
            x = block(x, mask)
        logits = self.lm_head(self.norm(x))
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

# 对比两个模型
v = 100
gpt = MiniGPT(v, model_dim=64, n_heads=4, n_layers=2)
llama = MiniLLaMA(v, model_dim=64, n_heads=8, n_kv_heads=2, n_layers=2)
print(f"原版 MiniGPT  参数量: {sum(p.numel() for p in gpt.parameters()):,}")
print(f"MiniLLaMA     参数量: {sum(p.numel() for p in llama.parameters()):,}")

x = torch.randint(0, v, (2, 10))
print(f"\nMiniLLaMA 输出 shape: {llama(x)[0].shape}")

---

## Part 3: 生成与推理

## 9. 自回归生成策略

4 种常见的生成方式：

| 策略 | 特点 | 适用场景 |
|------|------|----------|
| Greedy | 每次选概率最大的 | 确定性任务（代码、数学）|
| Temperature | 调节分布锐度 | 通用 |
| Top-K | 只从前 K 个候选中采样 | 平衡多样性和质量 |
| Top-P (Nucleus) | 从累积概率达 P 的候选中采样 | 自适应候选数量 |

In [ ]:
# === 4 种生成策略 ===

# 先训练一个小模型用于演示
text = "天地玄黄宇宙洪荒日月盈昃辰宿列张寒来暑往秋收冬藏闰余成岁律吕调阳云腾致雨露结为霜金生丽水玉出昆冈剑号巨阙珠称夜光果珍李柰菜重芥姜海咸河淡鳞潜羽翔"
chars = sorted(set(text))
vocab_size = len(chars)
c2i = {c: i for i, c in enumerate(chars)}
i2c = {i: c for c, i in c2i.items()}
data = torch.tensor([c2i[c] for c in text])

seq_len = 16
xs = torch.stack([data[i:i+seq_len] for i in range(len(data)-seq_len)])
ys = torch.stack([data[i+1:i+seq_len+1] for i in range(len(data)-seq_len)])

model = MiniLLaMA(vocab_size, model_dim=64, n_heads=8, n_kv_heads=2, n_layers=2, max_seq_len=seq_len+1)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
for ep in range(300):
    _, loss = model(xs, ys)
    opt.zero_grad(); loss.backward(); opt.step()
    if (ep+1) % 100 == 0:
        print(f"Epoch {ep+1} | Loss: {loss.item():.4f}")

# 生成函数
def generate(model, start, max_new=20, strategy='greedy', temperature=1.0, top_k=0, top_p=0.0):
    model.eval()
    idx = torch.tensor([[c2i[c] for c in start]])
    with torch.no_grad():
        for _ in range(max_new):
            logits, _ = model(idx[:, -model.max_seq_len:])
            logits = logits[:, -1, :] / temperature  # temperature 缩放

            if top_k > 0:  # Top-K: 只保留前 K 个
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, -1:]] = float('-inf')

            if top_p > 0:  # Top-P: 累积概率截断
                sorted_logits, sorted_idx = torch.sort(logits, descending=True)
                cumprobs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                mask = cumprobs - F.softmax(sorted_logits, dim=-1) >= top_p
                sorted_logits[mask] = float('-inf')
                logits = sorted_logits.scatter(1, sorted_idx, sorted_logits)

            if strategy == 'greedy':
                next_id = logits.argmax(dim=-1, keepdim=True)
            else:
                probs = F.softmax(logits, dim=-1)
                next_id = torch.multinomial(probs, 1)
            idx = torch.cat([idx, next_id], dim=1)
    return ''.join(i2c.get(i, '?') for i in idx[0].tolist())

# 对比 4 种策略
start = "天地"
print(f"=== 从 '{start}' 开始生成 ===\n")
print(f"Greedy:      {generate(model, start, strategy='greedy')}")
print(f"Temp=0.5:    {generate(model, start, strategy='sample', temperature=0.5)}")
print(f"Temp=1.5:    {generate(model, start, strategy='sample', temperature=1.5)}")
print(f"Top-K=5:     {generate(model, start, strategy='sample', top_k=5)}")
print(f"Top-P=0.9:   {generate(model, start, strategy='sample', top_p=0.9)}")

## 10. KV Cache

**问题**：自回归生成时，每生成一个 token 都要重新计算所有历史 token 的 K/V，浪费算力。

**解决**：缓存已计算的 K/V，每步只计算新 token 的 Q/K/V。

```
无 Cache: 生成第 n 个 token 时，计算 n 次 K/V  → O(n²) 总计算量
有 Cache: 生成第 n 个 token 时，只算 1 次 K/V  → O(n) 总计算量
```

In [ ]:
# === KV Cache 实现 ===

class GQA_WithCache(nn.Module):
    """带 KV Cache 的 GQA"""
    def __init__(self, model_dim, n_heads, n_kv_heads, max_seq_len=512):
        super().__init__()
        self.n_heads = n_heads
        self.n_kv_heads = n_kv_heads
        self.n_rep = n_heads // n_kv_heads
        self.head_dim = model_dim // n_heads
        self.Wq = nn.Linear(model_dim, n_heads * self.head_dim, bias=False)
        self.Wk = nn.Linear(model_dim, n_kv_heads * self.head_dim, bias=False)
        self.Wv = nn.Linear(model_dim, n_kv_heads * self.head_dim, bias=False)
        self.Wo = nn.Linear(model_dim, model_dim, bias=False)
        cos, sin = precompute_rope_freqs(self.head_dim, max_seq_len)
        self.register_buffer('cos', cos)
        self.register_buffer('sin', sin)

    def forward(self, x, start_pos, mask=None, kv_cache=None):
        B, T, _ = x.shape
        q = self.Wq(x).view(B, T, self.n_heads, self.head_dim).transpose(1, 2)
        k = self.Wk(x).view(B, T, self.n_kv_heads, self.head_dim).transpose(1, 2)
        v = self.Wv(x).view(B, T, self.n_kv_heads, self.head_dim).transpose(1, 2)

        # RoPE: 用 start_pos 偏移，确保位置正确
        cos = self.cos[start_pos:start_pos+T].unsqueeze(0).unsqueeze(0)
        sin = self.sin[start_pos:start_pos+T].unsqueeze(0).unsqueeze(0)
        q1, q2 = q[..., :self.head_dim//2], q[..., self.head_dim//2:]
        q = torch.cat([q1*cos - q2*sin, q1*sin + q2*cos], dim=-1)
        k1, k2 = k[..., :self.head_dim//2], k[..., self.head_dim//2:]
        k = torch.cat([k1*cos - k2*sin, k1*sin + k2*cos], dim=-1)

        # KV Cache: 拼接历史
        if kv_cache is not None:
            k = torch.cat([kv_cache[0], k], dim=2)
            v = torch.cat([kv_cache[1], v], dim=2)
        new_cache = (k, v)

        # 扩展 KV 头
        k_exp = k.repeat_interleave(self.n_rep, dim=1)
        v_exp = v.repeat_interleave(self.n_rep, dim=1)

        scores = torch.matmul(q, k_exp.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        out = torch.matmul(F.softmax(scores, dim=-1), v_exp)
        out = out.transpose(1, 2).contiguous().view(B, T, -1)
        return self.Wo(out), new_cache

class LLaMABlock_WithCache(nn.Module):
    def __init__(self, model_dim, n_heads, n_kv_heads, max_seq_len=512):
        super().__init__()
        self.norm1 = RMSNorm(model_dim)
        self.attn = GQA_WithCache(model_dim, n_heads, n_kv_heads, max_seq_len)
        self.norm2 = RMSNorm(model_dim)
        self.ffn = SwiGLU_FFN(model_dim)

    def forward(self, x, start_pos, mask=None, kv_cache=None):
        h, new_cache = self.attn(self.norm1(x), start_pos, mask, kv_cache)
        x = x + h
        x = x + self.ffn(self.norm2(x))
        return x, new_cache

class MiniLLaMA_WithCache(nn.Module):
    def __init__(self, vocab_size, model_dim=64, n_heads=8, n_kv_heads=2,
                 n_layers=2, max_seq_len=128):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, model_dim)
        self.blocks = nn.ModuleList([
            LLaMABlock_WithCache(model_dim, n_heads, n_kv_heads, max_seq_len)
            for _ in range(n_layers)
        ])
        self.norm = RMSNorm(model_dim)
        self.lm_head = nn.Linear(model_dim, vocab_size, bias=False)

    def forward(self, idx, start_pos=0, kv_caches=None):
        B, T = idx.shape
        x = self.tok_emb(idx)
        # Prefill 用因果 mask，decode 阶段不需要 mask（只有1个token，能看到所有cache）
        if T > 1:
            mask = torch.tril(torch.ones(T, T, device=idx.device)).unsqueeze(0).unsqueeze(0)
        else:
            mask = None
        new_caches = []
        for i, block in enumerate(self.blocks):
            cache = kv_caches[i] if kv_caches else None
            x, new_cache = block(x, start_pos, mask, cache)
            new_caches.append(new_cache)
        logits = self.lm_head(self.norm(x))
        return logits, new_caches

# === 速度对比 ===
def generate_no_cache(model_nc, start_ids, n=50):
    idx = start_ids.unsqueeze(0)
    with torch.no_grad():
        for _ in range(n):
            logits, _ = model_nc(idx)
            idx = torch.cat([idx, logits[:, -1:].argmax(-1)], dim=1)
    return idx[0]

def generate_with_cache(model_c, start_ids, n=50):
    # Prefill
    idx = start_ids.unsqueeze(0)
    with torch.no_grad():
        logits, caches = model_c(idx, start_pos=0)
        next_id = logits[:, -1:].argmax(-1)
        all_ids = [idx, next_id]
        pos = idx.shape[1]
        for _ in range(n - 1):
            logits, caches = model_c(next_id, start_pos=pos, kv_caches=caches)
            next_id = logits[:, -1:].argmax(-1)
            all_ids.append(next_id)
            pos += 1
    return torch.cat(all_ids, dim=1)[0]

# 构建两个模型（共享权重来公平对比）
v_size = 200
m_nc = MiniLLaMA(v_size, model_dim=64, n_heads=8, n_kv_heads=2, n_layers=4, max_seq_len=256)
m_c = MiniLLaMA_WithCache(v_size, model_dim=64, n_heads=8, n_kv_heads=2, n_layers=4, max_seq_len=256)

# 复制权重
m_c.tok_emb.load_state_dict(m_nc.tok_emb.state_dict())
m_c.norm.load_state_dict(m_nc.norm.state_dict())
m_c.lm_head.load_state_dict(m_nc.lm_head.state_dict())
for i in range(4):
    m_c.blocks[i].norm1.load_state_dict(m_nc.blocks[i].norm1.state_dict())
    m_c.blocks[i].norm2.load_state_dict(m_nc.blocks[i].norm2.state_dict())
    m_c.blocks[i].ffn.load_state_dict(m_nc.blocks[i].ffn.state_dict())
    m_c.blocks[i].attn.Wq.load_state_dict(m_nc.blocks[i].attn.Wq.state_dict())
    m_c.blocks[i].attn.Wk.load_state_dict(m_nc.blocks[i].attn.Wk.state_dict())
    m_c.blocks[i].attn.Wv.load_state_dict(m_nc.blocks[i].attn.Wv.state_dict())
    m_c.blocks[i].attn.Wo.load_state_dict(m_nc.blocks[i].attn.Wo.state_dict())

m_nc.eval(); m_c.eval()
start = torch.randint(0, v_size, (5,))
gen_len = 100

t0 = time.time()
out1 = generate_no_cache(m_nc, start, gen_len)
t_no = time.time() - t0

t0 = time.time()
out2 = generate_with_cache(m_c, start, gen_len)
t_yes = time.time() - t0

print(f"生成 {gen_len} tokens:")
print(f"  无 Cache: {t_no:.4f}s")
print(f"  有 Cache: {t_yes:.4f}s")
print(f"  加速比:   {t_no/t_yes:.2f}x")
print(f"  结果一致: {torch.equal(out1, out2)}")

---

## Part 4: 连接 MiniOneRec

## 11. 从 GPT 到推荐：SID 生成

MiniOneRec 的核心思想：**把推荐问题变成"生成下一个 token"问题**。

### 普通 GPT vs MiniOneRec

| | 普通 GPT | MiniOneRec |
|---|---------|------------|
| 词表 | 自然语言 token（几万个）| SID token（几百~几千）|
| 输入 | 文本 token 序列 | 用户历史 SID 序列 |
| 输出 | 下一个文字 token | 下一个物品 SID |
| 训练 | NTP on 文本 | NTP on SID 序列（SFT）|
| 生成 | 自由生成 | **约束解码**（只能生成合法 SID）|

### 约束解码（LogitProcessor）

```python
# 普通生成：从整个词表采样
next_token = sample(logits)  # 可能生成任何 token

# 约束解码：只允许合法的 SID token
valid_mask = get_valid_sids(prefix)  # 根据已生成的前缀，确定哪些 SID 合法
logits[~valid_mask] = -inf           # 屏蔽非法 token
next_token = sample(logits)          # 只能生成合法 SID
```

### MiniOneRec 的三阶段

```
阶段1: SID 构建
  物品文本 → Encoder → 嵌入 → RQ-VAE → SID (如 [12, 45, 78])

阶段2: SFT（就是我们学的 NTP！）
  用户历史 [SID_1, SID_2, ...] → GPT → 预测下一个 SID

阶段3: RL（GRPO）
  用 ranking reward 进一步优化生成质量
```

**关键洞察**：Step 3 学的 MiniGPT 训练和生成，就是 MiniOneRec SFT 阶段的核心！

## 11.5 DeepSeek MLA: Multi-Head Latent Attention

GQA 通过减少 KV 头数来压缩 KV Cache，但 DeepSeek V2/V3 提出了一种更激进的方案：**MLA（Multi-head Latent Attention）**。

### 核心思想：低秩压缩 KV

GQA 的思路是"少存几份 KV"，MLA 的思路是"把每份 KV 压缩到更小"。

```
GQA:  缓存 n_kv_heads 组完整的 K, V     → Cache 大小 = n_kv_heads × head_dim × 2
MLA:  缓存 1 个低秩压缩向量 c_kv        → Cache 大小 = d_c（远小于上面）
      推理时从 c_kv 还原出所有头的 K, V
```

### 对比

| | MHA | GQA | MLA |
|---|-----|-----|-----|
| KV Cache 存什么 | 所有头的 K, V | n_kv_heads 组 K, V | **1 个压缩向量 c_kv** |
| Cache 大小/token | 2 × H × d_h | 2 × G × d_h | **d_c**（如 512） |
| 还原方式 | 不需要 | repeat K/V | 线性投影还原 |
| 代表模型 | GPT-2 | LLaMA-2/3, Qwen | DeepSeek V2/V3 |

### 原理图解

```
标准 MHA/GQA:
  hidden → Wk → K (缓存这个，维度大)
  hidden → Wv → V (缓存这个，维度大)

MLA:
  hidden → W_dkv → c_kv (压缩，维度小，只缓存这个！)
                    ↓
              c_kv → W_uk → K (推理时还原)
              c_kv → W_uv → V (推理时还原)
```

**本质**：把 `hidden → K` 拆成 `hidden → c_kv → K`，中间的 `c_kv` 维度远小于 K，所以缓存 `c_kv` 更省显存。这是一个经典的**低秩分解**技巧（类似 LoRA 的思想）。

### RoPE 兼容问题

RoPE 需要直接作用在 K 上，但 MLA 的 K 是从压缩向量还原的，两者不兼容。DeepSeek 的解决方案：

```
K = [K_rope, K_nope]
     ↑          ↑
  单独投影     从 c_kv 还原
  (带 RoPE)   (不带 RoPE)
```

把 K 拆成两部分：一小部分专门用于 RoPE（单独缓存，很小），其余部分走低秩压缩。

### 面试一句话总结

> **GQA 是"少存几份 KV"，MLA 是"把 KV 压缩后再存"。MLA 用低秩投影将 KV Cache 压缩到一个小向量，推理时再还原，显存节省更激进，但计算略多。**

## 12. 模型对比总结

| 特性 | GPT-2 | LLaMA 2/3 | Qwen 2.5 | DeepSeek V3 |
|------|-------|-----------|-----------|-------------|
| Norm | LayerNorm | RMSNorm | RMSNorm | RMSNorm |
| FFN | ReLU | SwiGLU | SwiGLU | SwiGLU + MoE |
| Attention | MHA | GQA | GQA | MLA |
| 位置编码 | 学习式 | RoPE | RoPE | RoPE |
| Norm 位置 | Post-Norm | Pre-Norm | Pre-Norm | Pre-Norm |
| Bias | 有 | 无 | 无 | 无 |

### 面试要点

1. **为什么 Decoder-Only？** 统一的自回归框架，scaling law 更好，生成任务天然适配
2. **NTP 为什么有效？** 预测下一个 token 隐含了对语言/序列的深层理解
3. **RMSNorm vs LayerNorm？** 去掉均值计算，更快，效果相当
4. **SwiGLU vs ReLU？** 门控机制提供更强非线性，实验效果更好
5. **GQA 的意义？** 减少 KV Cache 显存，推理更快，效果接近 MHA
6. **KV Cache 原理？** 缓存历史 K/V 避免重复计算，从 O(n²) 降到 O(n)
7. **MLA vs GQA？** GQA 减少 KV 头数，MLA 用低秩压缩把 KV 压到一个小向量再缓存，显存节省更激进；RoPE 通过拆分 K 为 rope/nope 两部分解决兼容问题
8. **约束解码？** 在 logits 上 mask 非法 token，保证生成合法输出

## 13. 下一步

- [ ] 把 Step 3 学习笔记补到 `study.md`
- [ ] 进入 Step 4：阅读 MiniOneRec 的 `sft.py`，对照本节学的 NTP 训练
- [ ] 进入 Step 5：阅读 `rl.py`，理解 GRPO 如何在 NTP 基础上优化

---

> **Step 3 完成！** 你现在理解了：
> 1. Decoder-Only GPT 的结构和 NTP 训练
> 2. LLaMA 的三大改动：RMSNorm、SwiGLU、GQA
> 3. 4 种生成策略 + KV Cache 加速
> 4. MiniOneRec 如何把推荐变成 token 生成问题